# Rework project description column

Goal is to add to the description of projects with multiple funding programs. Current format just picks one.

In [1]:
import _data_utils
import _script_utils
import numpy as np
import pandas as pd
from calitp_data_analysis.sql import to_snakecase
from siuba import *

In [2]:
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_colwidth", None)

In [3]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/dla/dla-iija"

In [4]:
GCS_FILE_NAME = "FMIS_Projects_Universe_IIJA_Reporting_062923_ToDLA.xlsx"

### Testing script updated to this file from 6-29-2023

In [5]:
locodes = to_snakecase(
    pd.read_excel(
        f"gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/locodes_updated7122021.xlsx"
    )
)

In [6]:
locodes.head(2)

,agency_locode,agency_name,district,county_name,rtpa_name,mpo_name,mpo_locode_fads,active_e76s______7_12_2021_
0,6302,Humboldt Bay Harbor Recreation & Conservation District,1,Humboldt County,Humboldt County Association of Governments,NON-MPO,NON-MPO,NaN
1,6330,Willow Creek Community Services District,1,Humboldt County,Humboldt County Association of Governments,NON-MPO,NON-MPO,NaN


In [7]:
new_locodes = to_snakecase(
    pd.read_csv(f"{GCS_FILE_PATH}/program_codes/LP2000_agencies_view.csv")
)

In [8]:
new_locodes

,agency_code,agency_name,agencyid,last_update
0,5001,San Francisco,City of San Francisco,11/9/2023 4:59am
1,5002,Sacramento,City of Sacramento,11/9/2023 4:59am
2,5003,Benicia,City of Benicia,11/9/2023 4:59am
3,5004,San Diego,City of San Diego,11/9/2023 4:59am
4,5005,San Jose,City of San Jose,11/9/2023 4:59am
...,...,...,...,...
915,7502,Hoopa Valley Tribe,"""Hoopa Valley Tribe, California""",11/9/2023 4:59am
916,7503,San Diego Metropolitan Transit System,San Diego Metropolitan Transit System,11/9/2023 4:59am
917,7504,San Pasqual Band of Mission Indians,San Pasqual Band of Diegueno Mission Indians of California,11/9/2023 4:59am
918,7505,Gold Coast Transit District,Gold Coast Transit District,11/9/2023 4:59am


In [9]:
all_locodes = pd.merge(
    locodes,
    new_locodes,
    left_on="agency_locode",
    right_on="agency_code",
    how="outer",
    indicator=True,
)

In [10]:
all_locodes.loc[all_locodes._merge != "both"][
    [
        "agency_locode",
        "agency_name_x",
        "agency_code",
        "agency_name_y",
        "agencyid",
        "_merge",
    ]
]

,agency_locode,agency_name_x,agency_code,agency_name_y,agencyid,_merge
21,6201.0,Caltrans,NaN,NaN,NaN,left_only
26,6063.0,Eureka Southern Railroad Company-DO NOT USE see 6242,NaN,NaN,NaN,left_only
29,6400.0,Lake County Air Basin,NaN,NaN,NaN,left_only
33,6487.0,Humboldt County Resource Conservation,NaN,NaN,NaN,left_only
34,6500.0,Mendocino County Health and Human Services Agency,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...
1039,6176.0,Tri-Counties Regional Park Group,NaN,NaN,NaN,left_only
1040,6185.0,Trust for Public Land,NaN,NaN,NaN,left_only
1041,NaN,NaN,7504.0,San Pasqual Band of Mission Indians,San Pasqual Band of Diegueno Mission Indians of California,right_only
1042,NaN,NaN,7505.0,Gold Coast Transit District,Gold Coast Transit District,right_only


In [11]:
no_match_x = all_locodes.loc[all_locodes._merge == "left_only"][
    ["agency_locode", "agency_name_x"]
]

In [12]:
no_match_y = all_locodes.loc[all_locodes._merge == "left_only"][
    ["agency_locode", "agency_name_y"]
]

In [13]:
second_match = pd.merge(
    no_match_x,
    no_match_y,
    left_on="agency_name_x",
    right_on="agency_name_y",
    how="outer",
    indicator=True,
)

#### Checking in on names
* The names that did not match from the new list are not present in the old list. More names from the old list did not make it over to the  new list...

In [14]:
# df_all = (pd.merge(df, locodes, left_on='locode', right_on='agency_locode', how='left'))

In [15]:
df = _script_utils._prep_data(GCS_FILE_NAME)

In [16]:
df.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,iija_program_code,funding_type_code,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
921,2022-09-08,YS30,Highway Safety Improvement Program (HSIP),0052036,1218000102S,"IN ORANGE COUNTY IN ANAHEIM AT KATELLA AVENUE UNDERCROSSING COLD PLANE, PLACE RHMA & HSFT, REPLACE SAFETY DEVICES AND SIGNS",59,Cong Dist 46,Active,"ON STATE ROUTE: 5. IN ORANGE COUNTY IN ANAHEIM AT KATELLA AVENUE UNDERCROSSING COLD PLANE, PLACE RHMA & HSFT, REPLACE SAFETY DEVICES AND SIGNS",17,Construction Engineering,701000.0,641900.0,S SCAG,YS30,IIJA-F,NaN,California,12.0,Orange County,NaN,NaN


### New Program Codes

Now in script

In [17]:
df.loc[df.program_code_description.isna()].program_code.nunique()

0

In [18]:
## comparing codes:

code1 = to_snakecase(
    pd.read_excel(
        f"{GCS_FILE_PATH}/program_codes/FY21-22ProgramCodesAsOf5-25-2022.v2_expanded090823.xlsx"
    )
)[["iija_program_code", "new_description"]]

In [19]:
code2 = to_snakecase(
    pd.read_excel(f"{GCS_FILE_PATH}/program_codes/Copy of lst_IIJA_Code_20230908.xlsx")
)[["iija_program_code", "description", "program_name"]]

In [20]:
code_merge = pd.merge(code1, code2, on="iija_program_code", how="outer", indicator=True)

In [21]:
code_merge["new_description"] = code_merge["new_description"].str.strip()

In [22]:
compare_col = np.where(
    code_merge["new_description"] == code_merge["program_name"], True, False
)

In [23]:
code_merge["compare_descriptions_to_programname"] = compare_col
code_merge.compare_descriptions_to_programname.value_counts()

True     79
False    57
Name: compare_descriptions_to_programname, dtype: int64

In [24]:
compare_col2 = np.where(
    code_merge["new_description"] == code_merge["description"], True, False
)
code_merge["compare_descriptions_cols"] = compare_col2
code_merge.compare_descriptions_cols.value_counts()

True     87
False    49
Name: compare_descriptions_cols, dtype: int64

In [25]:
code_merge.loc[
    (code_merge.compare_descriptions_cols == False)
    & (code_merge._merge == "both")
    & (code_merge.compare_descriptions_to_programname == False)
].head()

,iija_program_code,new_description,description,program_name,_merge,compare_descriptions_to_programname,compare_descriptions_cols
1,Y002,National Highway Performance Program (NHPP),NHPP Exempt,NHPP Exempt,both,False,False
13,Y130,National Electric Vehicle Infrastructure (NEVI) Program,National Electrict Vehicle Formula Program*Amended 2/1/22,National Electrict Vehicle Formula Program,both,False,False
16,Y230,Surface Transportation Block Grant (STBG) Program,STBG Program - Urbanized Areas With Population Over 200K,Surface Transportation Block Grant,both,False,False
17,Y233,Surface Transportation Block Grant (STBG) Program,STBG Program Off-System Bridge,Surface Transportation Block Grant,both,False,False
19,Y235,"Special Rule for Areas of 5,000 to Less Than 50,000 Population","Special Rule for Areas of 5,000 to Less Than 50,000 Population [NEW]",Special Rule,both,False,False


In [26]:
code_merge.loc[code_merge._merge != "both"].head()

,iija_program_code,new_description,description,program_name,_merge,compare_descriptions_to_programname,compare_descriptions_cols
133,ER01,NaN,Emergency Suppliment Funding,Emergency Supplement Funding,right_only,False,False
134,Y520,NaN,"Federal Lands Planning Program, Access Planning",Federal Lands,right_only,False,False
135,Y928,NaN,Community Project Funding Congressionally Directed Spending,Community Project Funding Congressionally Directed Spending,right_only,False,False


In [27]:
code_merge.new_description.fillna(code_merge["description"], inplace=True)

In [28]:
code_merge = code_merge.drop(
    columns={
        "description",
        "_merge",
        "compare_descriptions_cols",
        "compare_descriptions_to_programname",
    }
)

In [29]:
code_merge.head()

,iija_program_code,new_description,program_name
0,Y001,National Highway Performance Program (NHPP),National Highway Performance Program (NHPP)
1,Y002,National Highway Performance Program (NHPP),NHPP Exempt
2,Y003,Projects to Reduce PM 2.5 Emissions,Projects to Reduce PM 2.5 Emissions
3,Y008,Protective Features,Protective Features
4,Y030,Redistribution of Certain Authorized Funds,Redistribution of Certain Authorized Funds


In [30]:
code_merge.loc[code_merge.program_name.str.contains("Program")]

,iija_program_code,new_description,program_name
0,Y001,National Highway Performance Program (NHPP),National Highway Performance Program (NHPP)
10,Y110,Bridge Formula Program,Bridge Formula Program
11,Y120,Bridge Formula Program,Bridge Formula Program
12,Y120,Bridge Formula Program,Bridge Formula Program
13,Y130,National Electric Vehicle Infrastructure (NEVI) Program,National Electrict Vehicle Formula Program
49,Y444,Highway Research and Development Program,Highway Research and Development Program
50,Y446,Highway Research and Development Program,Highway Research and Development Program
51,Y448,Highway Research and Deveopment Program,Highway Research and Development Program
53,Y450,Metropolitan Planning Program,Metropolitan Planning Program
54,Y460,National Highway Freight Program (NHFP),National Highway Freight Program (NHFP)


### Add new program codes to the data

In [31]:
agg1 = df.groupby(["project_number"]).agg({"program_code": "nunique"})

In [32]:
agg1.loc[agg1.program_code > 1].sort_values(by=["program_code"])

,program_code
project_number,
000C524,2
5928128,2
5928125,2
5925176,2
5925162,2
...,...
0010209,3
6066137,3
5002200,3


In [33]:
agg1.loc[agg1.program_code > 2].sort_values(by=["program_code"]).head()

,program_code
project_number,
0010203,3
0010206,3
0010209,3
00PE023,3
5002200,3


In [34]:
## take a look at those 9 projects

morethantwopc = (
    df
    >> group_by(_.project_number, _.implementing_agency, _.project_title)
    >> summarize(n=_.program_code_description.nunique())
    >> filter(_.n > 2)
    >> arrange(-_.n)
)

In [35]:
morethantwopc = (
    df.groupby(["project_number", "implementing_agency", "project_title"])
    .agg({"program_code_description": "nunique"})
    .reset_index()
)

In [36]:
morethantwopc = morethantwopc.loc[
    morethantwopc.program_code_description > 2
].sort_values(by=["program_code_description"])

In [37]:
agg1 = df.groupby(["project_number"]).agg({"program_code": "nunique"})

In [38]:
project_numbers_to_check = morethantwopc.project_number.to_list()

In [39]:
project_numbers_to_check

['0010203',
 '0010206',
 '0010209',
 '00PE023',
 '5002203',
 '5004049',
 '5942283',
 '6066137',
 '6054094']

In [40]:
## work on agg script

In [41]:
full = _script_utils.get_clean_data(df, full_or_agg="full")

Index(['fmis_transaction_date', 'project_number', 'implementing_agency',
       'summary_recipient_defined_text_field_1_value', 'funding_type_code',
       'program_code', 'program_code_description', 'recipient_project_number',
       'improvement_type', 'improvement_type_description',
       'program_code_description_for_description', 'project_title',
       'obligations_amount', 'total_cost_amount', 'congressional_district',
       'district', 'county_code', 'county_name', 'county_name_abbrev',
       'county_name_title', 'implementing_agency_locode', 'rtpa_name',
       'mpo_name'],
      dtype='object')

In [42]:
full["project_title"] = full.project_title.str.title()
full["project_description"] = full.project_description.str.title()

In [43]:
full.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,iija_program_code,funding_type_code,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,county_name_abbrev,county_name_title,program_code_description_for_description,new_project_title
2533,2023-03-29,ER01,Emergency Supplement Funding,39B4001,0419000351S,"In San Mateo County, Near Montara, At 0.9 Miles South Of Devil'S Slide Tunnel Construct Secant Pile Tie-Back Wall",81,14,Closed,"In San Mateo County, Near Montara, At 0.9 Miles South Of Devil'S Slide Tunnel Construct Secant Pile Tie-Back Wall",17,Construction Engineering,1199932.23,1062300.0,S ER NONE,ER01,IIJA-A,nan,California,04,San Mateo County,NaN,NaN,SM,San Mateo County,Emergency Supplement Funding,Road Restoration & Rehabilitation in San Mateo County


In [44]:
## testing the project code descriptions grouped
test_pc = (
    full.assign(count=1)
    .groupby("project_number")
    .agg({"program_code_description": lambda x: ", and the ".join(x.unique())})
    .reset_index()
)

In [45]:
test_pc[test_pc["project_number"].isin(project_numbers_to_check)]

,project_number,program_code_description
38,0010203,"Safe and Accessible Transportation Options - Metropolitan Planning Program, and the Metropolitan Planning Program, and the State Planning and Research Program"
41,0010206,"Safe and Accessible Transportation Options - Metropolitan Planning Program, and the Metropolitan Planning Program, and the State Planning and Research Program"
44,0010209,"Safe and Accessible Transportation Options - Metropolitan Planning Program, and the Metropolitan Planning Program, and the State Planning and Research Program"
93,00PE023,"National Highway Performance Program (NHPP), and the Surface Transportation Block Grant Program, and the Highway Safety Improvement Program (HSIP)"
495,5002203,"Surface Transportation Block Grant Program, and the Transportation Alternatives Program, and the Congestion Mitigation & Air Quality Improvement Program"
506,5004049,"National Highway Performance Program (NHPP), and the Bridge Formula Program, and the Bridge Replacement and Rehabilitation Program"
1034,5942283,"Projects to Reduce PM 2.5 Emissions Program, and the Surface Transportation Block Grant Program, and the Congestion Mitigation & Air Quality Improvement Program"
1115,6054094,"Surface Transportation Block Grant Program, and the Congestion Mitigation & Air Quality Improvement Program, and the Carbon Reduction Program, and the Community Project Funding Congressionally Directed Spending Program"
1121,6066137,"Surface Transportation Block Grant Program, and the Transportation Alternatives Program, and the Carbon Reduction Program"


## Test agg

In [46]:
agg = _script_utils.get_clean_data(df, full_or_agg="agg")

Index(['fmis_transaction_date', 'project_number', 'implementing_agency',
       'summary_recipient_defined_text_field_1_value', 'funding_type_code',
       'program_code', 'program_code_description', 'recipient_project_number',
       'improvement_type', 'improvement_type_description',
       'program_code_description_for_description', 'project_title',
       'obligations_amount', 'total_cost_amount', 'congressional_district',
       'district', 'county_code', 'county_name', 'county_name_abbrev',
       'county_name_title', 'implementing_agency_locode', 'rtpa_name',
       'mpo_name'],
      dtype='object')

/home/jovyan/data-analyses/dla/iija/_script_utils.py:621: FutureWarning: The default value of regex will change from True to False in a future version.
  df['implementing_agency_locode'] = df['implementing_agency_locode'].str.replace('.0', '')


True

In [47]:
agg.sample(5)

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,funding_type_code,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,old_project_title_desc,obligations_amount,total_cost_amount,congressional_district,district,county_code,county_name,county_name_abbrev,implementing_agency_locode,rtpa_name,mpo_name,new_project_title,new_description_col
873,2023-03-21,5392059,Thousand Oaks,L5392SCAG,IIJA-F,YS70,Vulnerable Road User Safety Special Rule Program,0719000328L,17|28,Construction Engineering|Facilities for Pedestrians and Bicycles,"THOUSAND OAKS BLVD., BETWEEN CONEJO SCHOOL ROAD AND SKYLINE DRIVE RELOCATION AND CONSOLIDATION OF TWO EXISTING CROSSWALKS LOCATED ON THOUSAND OAKS BL",414540,652190,|26|,|07|,111,Ventura County,|VEN|,5392,Ventura County Transportation Commission,Southern California Association Of Governments,Facilities for Pedestrians and Bicycles in Thousand Oaks,"Facilities for Pedestrians and Bicycles in Thousand Oaks, part of the Vulnerable Road User Safety Special Rule Program. (Federal Project ID: 5392059)."
1305,2023-06-20,0010204,California,S TAHOE,IIJA-F,Y410|Y450,Safe and Accessible Transportation Options - Metropolitan Planning Program|Metropolitan Planning Program,0022000349S,18,Planning,OWP 22/23 TAHOE REGION CONSOLIDATED PLANNING GRANT (CPG) PROJECT FOR TAHOE REGIONAL PLANNING AGENCY (TRPA) OVERALL WORK PROGRAM (OWP) FOR FISCAL YEAR,772657,772657,|01|,|nan|,999,Statewide,|NA|,nan,None,None,Planning and Research in Tahoe Regional Planning Agency,"Planning and Research in Tahoe Regional Planning Agency, part of the Safe and Accessible Transportation Options - Metropolitan Planning Program, and the Metropolitan Planning Program. (Federal Project ID: 0010204)."
1175,2023-05-31,5037037,Chico,L5037BCAG,IIJA-F,Y300|Y400,Transportation Alternatives Program|Congestion Mitigation & Air Quality Improvement Program,0317000328L,4|17,4R - No Added Capacity|Construction Engineering,THE ESPLANADE FROM 11TH AVE TO THE BIG CHICO CREEK BRIDGE AND INCLUDES OLEANDER AVENUE BETWEEN MEMORIAL AVE AND 10TH ST. ESPLANADE CORRIDOR SAFETY AN,6909722,8524936,|01|,|03|,7,Butte County,|BUT|,37,Butte County Association of Governments,Butte County Association Of Governments,Road Construction in Chico,"Road Construction in Chico, part of the Transportation Alternatives Program, and the Congestion Mitigation & Air Quality Improvement Program. (Federal Project ID: 5037037)."
856,2023-03-15,6216003,San Francisco Public Utilities Commission,L6216MTC,IIJA-F,Y230,Surface Transportation Block Grant Program,0415000364L,17|28,Construction Engineering|Facilities for Pedestrians and Bicycles,EAST SIDE OF SR 35 BETWEEN THE INTERSECTION OF HWYY 92 & HWY 35 SOUTHWARD 6 MILES TO SFPUC PENINSULA WATERSHED PROPERTY NEW 6-10 FOOT WIDE MULTI-MODA,1000000,18890806,|14|,|04|,75,San Francisco County,|SF|,6216,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Facilities for Pedestrians and Bicycles in San Francisco Public Utilities Commission,"Facilities for Pedestrians and Bicycles in San Francisco Public Utilities Commission, part of the Surface Transportation Block Grant Program. (Federal Project ID: 6216003)."
102,2022-04-11,5037022,Chico,L5037BCAG,IIJA-F,Y233,Surface Transportation Block Grant Program,0300020680L,16,Right of Way,SALEM STREET AT LITTLE CHICO CREEK BRIDGE BR. # 12C-336 ~0.1 M NORTH OF 10TH ST BRIDGE REHABILITATION (TC),100000,100000,|01|,|03|,7,Butte County,|BUT|,37,Butte County Association of Governments,Butte County Association Of Governments,Bridge Rehabilitation in Chico,"Bridge Rehabilitation in Chico, part of the Surface Transportation Block Grant Program. (Federal Project ID: 5037022)."


In [48]:
agg[agg["project_number"].isin(project_numbers_to_check)]

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,funding_type_code,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,old_project_title_desc,obligations_amount,total_cost_amount,congressional_district,district,county_code,county_name,county_name_abbrev,implementing_agency_locode,rtpa_name,mpo_name,new_project_title,new_description_col
956,2023-04-19,5002203,Sacramento,L5002SACOG,IIJA-F,Y230|Y301|Y400,Surface Transportation Block Grant Program|Transportation Alternatives Program|Congestion Mitigation & Air Quality Improvement Program,0318000098L,4|17,4R - No Added Capacity|Construction Engineering,ON BROADWAY FROM 3RD STREET TO 16TH STREET (BROADWAY COMPLETE STREETS PHASE 1 PROJECT) & FROM 16TH STREET TO 24TH STREET (BROADWAY COMPLETE STEETS PH,10903000,12317000,|06|,|03|,67,Sacramento County,|SAC|,02,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,Complete Streets in Sacramento,"Complete Streets in Sacramento, part of the Surface Transportation Block Grant Program, and the Transportation Alternatives Program, and the Congestion Mitigation & Air Quality Improvement Program. (Federal Project ID: 5002203)."
1187,2023-06-01,6054094,Riverside County Transportation Commission,L6054SCAG,IIJA-F|IIJA-A,Y230|Y400|Y601|Y926,Surface Transportation Block Grant Program|Congestion Mitigation & Air Quality Improvement Program|Carbon Reduction Program|Community Project Funding Congressionally Directed Spending Program,0819000062L,15,Preliminary Engineering,I-15 FROM CAJALCO ROAD IN THE CITY OF CORONA TO STATE ROUTE 74 IN THE CITY OF LAKE ELSINORE EXTEND TOLL EXPRESS LANES SOUTH IN EACH TRAVEL DIRECTION,40151820,40151820,|42|,|08|,65,Riverside County,|RIV|,54,Riverside County Transportation Commission,Southern California Association Of Governments,Extend Express Lanes in Riverside County Transportation Commission,"Extend Express Lanes in Riverside County Transportation Commission, part of the Surface Transportation Block Grant Program, and the Congestion Mitigation & Air Quality Improvement Program, and the Carbon Reduction Program, and the Community Project Funding Congressionally Directed Spending Program. (Federal Project ID: 6054094)."
1188,2023-06-01,6066137,San Diego Association of Governments,L6066SANDAG,IIJA-F,Y230|Y301|Y601,Surface Transportation Block Grant Program|Transportation Alternatives Program|Carbon Reduction Program,1118000175L,17|28,Construction Engineering|Facilities for Pedestrians and Bicycles,ALONG IMPERIAL AVENUE FROM 21ST STREET TO 47TH STREET IN THE CITY OF SAN DIEGO. THE IMPERIAL AVENUE BIKEWAY IS A REGIONAL PRIORITY BIKEWAY CONNECTING,8906000,10032758,|51|,|11|,73,San Diego County,|SD|,66,NON-RTPA,San Diego Association Of Governments,Bike Lanes in San Diego Association of Governments,"Bike Lanes in San Diego Association of Governments, part of the Surface Transportation Block Grant Program, and the Transportation Alternatives Program, and the Carbon Reduction Program. (Federal Project ID: 6066137)."
1230,2023-06-14,00PE023,California,S NON-MPO,IIJA-F,Y001|Y240|YS30,National Highway Performance Program (NHPP)|Surface Transportation Block Grant Program|Highway Safety Improvement Program (HSIP),00000PE023S,15,Preliminary Engineering,STATEWIDE STATEWIDE PRELIMINARY ENGINEERING 2022-23,376813864,429983107,|01|,|nan|,999,Statewide,|NA|,nan,None,None,Preliminary Engineering Projects Statewide,"Preliminary Engineering Projects Statewide, part of the National Highway Performance Program (NHPP), and the Surface Transportation Block Grant Program, and the Highway Safety Improvement Program (HSIP). (Federal Project ID: 00PE023)."
1274,2023-06-16,5942283,Fresno County,L5942FCOG,IIJA-F,Y003|Y230|Y400,Projects to Reduce PM 2.5 Emissions Program|Surface Transportation Block Grant Program|Congestion Mitigation & Air Quality Improvement Program,0617000208L,4|17|16,4R - No Added Capacity|Constru

In [49]:
assert len(agg) == df.project_number.nunique()